In [2]:
# -*- coding:utf-8 -*-
import urllib.request
from urllib.parse import quote
import string
import json
import sys
import time
import pandas as pd
import numpy as np
from urllib.error import HTTPError
#import codecs
#import os
#import urllib2

In [3]:
#    def ls_row(self):  分区函数，输入一个区，分成9个区
def new_bounds(area):    
    
    #lat_sw = float(self.loc.split(',')[0])        #按逗号，分割字符串loc_all，取第一个
    #lat_ne = float(self.loc.split(',')[2])
    #lng_sw = float(self.loc.split(',')[1])
    #lng_ne = float(self.loc.split(',')[3])
    area = np.array(area)
    lat_sw = area[0]      #按逗号，分割字符串loc_all，取第一个
    lat_ne = area[2]
    lng_sw = area[1]
    lng_ne = area[3]
  
    new_bounds = []
        #  col_row 将bounds的每一小块继续细分为3行3列，可以防止区域内的搜索数量上限400
    col_row = 3 
        #for lst in bounds:
    distance_lat = (lat_ne - lat_sw)/col_row
    distance_lon = (lng_ne - lng_sw)/col_row
    for i in range(col_row):
        for j in range(col_row):
            lst_temp = []
            lst_temp.append(lat_sw+distance_lat*i)
            lst_temp.append(lng_sw+distance_lon*j)
            lst_temp.append(lat_sw+distance_lat*(i+1))
            lst_temp.append(lng_sw+distance_lon*(j+1))
            new_bounds.append(lst_temp)
        
    return new_bounds

In [4]:
def read_areas(filename): 
    #filename = ".\\area_segment.csv"       #  读取搜索区域
    f = open(filename,"rb")
    Lat_lng = pd.read_csv(f,header = None)
    f.close()

    areas = []
    for k in range(0,len(Lat_lng)):
        a = [Lat_lng.iloc[k][0],Lat_lng.iloc[k][1],Lat_lng.iloc[k][2],Lat_lng.iloc[k][3]]
        areas.append(a)
    return areas

In [5]:
def baidu_search(itemy,loc):
        
    json_sel = []        #搜索结果list  每一个区域搜索 返回的结果，写入csv
    page_size = 20
    page_num = 0
    coord_type = 3
    np = True
    a=[]
    i = 0      # ak 索引
    while np == True:
        total = 0
        url="http://api.map.baidu.com/place/v2/search?ak="+str(ak[i])+"&output=json&query="+str(itemy[0])+ \
                "&page_size="+str(page_size)+"&page_num="+str(page_num)+"&coord_type="+ str(coord_type) + \
                "&bounds="+str(loc[0])+","+str(loc[1])+","+str(loc[2])+","+str(loc[3])
            
            #url = 'http://api.map.baidu.com/place/v2/search?query=' + self.itemy + '&bounds=' + self.loc + '&page_size=20&page_num=' + \
           #     str(pages) + '&output=json&ak=' + api_key
            
        url = quote(url, safe=string.printable)
            
            #请求url读取，创建网页对象
            
        try:
            jsonf = urllib.request.urlopen(url)
        except HTTPError:
            print("HTTPError，第:",i+1,"个ak发生网络错误",ak[i] )
            i += 1
            if i >= len(ak):
                print("ak已用完，请明天再下载")
                sys._exit(0)            #退出，引发一个可捕获异常
            else:
                print("用第:", i+1,"个ak,继续下载", ak[i])
                continue
                
            
        page_num=page_num+1
        
            #判断查询翻页进程
        jsonfile=jsonf.read()
        s=json.loads(jsonfile)
           # print(s)
        total=int(s["total"])
            #total=int(s.get("total"))
        a.append(total)
            
            # for url in self.urls():        #
            #json_obj = urllib2.urlopen(url)
            #data = json.load(json_obj)
        for item in s['results']:
            jname = item["name"]
            jlat = item["location"]["lat"]
            jlng = item["location"]["lng"]
            js_sel = jname + ',' + str(jlat) + ',' + str(jlng)
            json_sel.append(js_sel)
                               
        max_page=int(a[0]/page_size)+1
            #防止并发过高，百度地图要求并发小于120
        time.sleep(1) 
        
        if page_num>max_page:
            np=False
            page_num=0
                
    print("search complete")
    print("output: ", str(loc))
    print("total: ", str(a[0]))
    print(json_sel)
    print("")
                
    return json_sel,a[0]
                
        
def write_poi(json_sel,filename):
    
    #filename = ".\\results.csv"
        #filename =[".\\POI\\a_stdout_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
    doc = open(filename, 'a+')
        #doc.write(codecs.BOM_UTF8)
        #for ax in json_sel:
        #    doc.write(ax)
        #    doc.write('\n')
    for item in range(0,len(json_sel)):
        doc.write(json_sel[item])
        doc.write("\n")
    doc.close()       
        

In [8]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a+')

    def write(self, message):
        self.terminal.write(message)
        self.terminal.flush()
        self.log.write(message)
        self.log.flush()

    def flush(self):
        pass

filename_3 =[".\\log\\a_stdout_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
#filename2 =[".\\log\\a_stderr_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log_file']
    
#sys.stdout = Logger(".\\a_stdout.log", sys.stdout)
#sys.stderr = Logger(".\\a_stderr.log_file", sys.stderr)             # redirect std err, if necessary
sys.stdout = Logger(filename_3[0], sys.stdout)
#sys.stderr = Logger(filename2[0], sys.stderr)    

In [9]:
# 9*9 区域搜索
if __name__ == '__main__':
    # ak
    #ak ="zmBxzD9D8mogOf8iQygQ5GTkMgKDgLT5"   # 这里填入你的百度API的ak
    filename_1 = ".\\area_segment.csv"     #输入文件
    filename_2 = ".\\results.csv"          #输出文件
    ak = [
                "GsHF3EoEFsrRWv3L8Rd3shGHnBO8Zb45",
        "vOvll2Da1Uy78aHfiTgTAOZ5mZpFFO2f",
        "5UnM9eSMEDbfsjKg1IobksqHqGe8vjKB",
        "TXolGK5eEl3syP09a479uq4GbWgTCp3p",
        "AMQKttYPG1XgO7pSDbil8Lshswuvdnr6",
        "XywGfVo4kXI65eRr110G9r8ZdgNlTZsW",
        "MKrwWMxOir7Qc8ozfKmcUQusjPn6F0mB",
        "hv5PSMqxB8S01UMWrL6bFrj0e0Ow30PM",
        "RCaptGZviM4Ayj5gVMdjEM4dOzsFnIX4",
        "lfN9m3QcUNrIIyx7GrA3trSBxPomRmW4",
        "zmBxzD9D8mogOf8iQygQ5GTkMgKDgLT5",
        "Mep63CoO93mgoD3XlEFQ6MdTkNNugYKg"
    ]
    itemy = ["超市$商店"]        #不同的关键字，以 $ 隔开
    
    #filename = ".\\area_segment.csv"       #  读取搜索区域
    #f = open(filename,"rb")
    #Lat_lng = pd.read_csv(f,header = None)
    #f.close()
    
    areas = read_areas(filename_1)

    start_time = time.time()
    print ("开始爬取数据，请稍等...", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    
    num_global = 0
   
    for j in range(0, len(areas)):
        #a = str(Lat_lng.iloc[item][0])+ "," + str(Lat_lng.iloc[item][1])+ ","+ str(Lat_lng.iloc[item][2])+","+str(Lat_lng.iloc[item][3])
        #Area = LocaDiv(a)
        a = areas[j]
        areas_to_use = new_bounds(a)
        print(areas_to_use)
        i = 0
        num_single = 0
        for area_to_use in areas_to_use:
            i += 1
            print("现在爬取第" ,j+1 ,"大区的第",i, "区域")
            #par = baidu_seach(area_to_use)  # 请修改爬取的类别
            [json_sel, total] = baidu_search(itemy, area_to_use)
            num_single += total
            write_poi(json_sel,filename_2)
            
        print("第",j+1 ,"大区共爬取POI", num_single,"个")
        print("")
        num_global += num_single
      
    end_time = time.time()
    print ("爬取完毕，用时: ", float(end_time - start_time)/60, " 分钟",time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )
    print("该项目共爬取POI",num_global,"个")
    print("")

KeyError: 'total'

In [ ]:
loc = [39.695599, 116.103444, 39.85816733, 116.3088807]        
itemy = "中学"
[json_sel, total] = baidu_search(itemy, loc)

print(json_sel, total)


In [10]:
#def baidu_search(itemy,loc):
loc = [39.695599, 116.103444, 39.85816733, 116.3088807]        
itemy = "中学"
json_sel = []        #搜索结果list  每一个区域搜索 返回的结果，写入csv
page_size = 20
page_num = 0
coord_type = 3
np = True
a=[]
i = 0      # ak 索引
while np == True:
    total = 0
    url="http://api.map.baidu.com/place/v2/search?ak="+str(ak[i])+"&output=json&query="+str(itemy[0])+ \
                "&page_size="+str(page_size)+"&page_num="+str(page_num)+"&coord_type="+ str(coord_type) + \
                "&bounds="+str(loc[0])+","+str(loc[1])+","+str(loc[2])+","+str(loc[3])
            
            #url = 'http://api.map.baidu.com/place/v2/search?query=' + self.itemy + '&bounds=' + self.loc + '&page_size=20&page_num=' + \
           #     str(pages) + '&output=json&ak=' + api_key
            
    url = quote(url, safe=string.printable)
            
            #请求url读取，创建网页对象
            
    try:
        jsonf = urllib.request.urlopen(url)
    except HTTPError:
        print("HTTPError，第:",i+1,"个ak发生网络错误",ak[i] )
        i += 1
        if i >= len(ak):
            print("ak已用完，请明天再下载")
            sys._exit(0)            #退出，引发一个可捕获异常
        else:
            print("用第:", i+1,"个ak,继续下载", ak[i])
            continue
                
            
    page_num=page_num+1
        
            #判断查询翻页进程
    jsonfile=jsonf.read()
    s=json.loads(jsonfile)
    print(s)
    total=int(s["total"])
            #total=int(s.get("total"))
    a.append(total)
            
            # for url in self.urls():        #
            #json_obj = urllib2.urlopen(url)
            #data = json.load(json_obj)
    for item in s['results']:
        jname = item["name"]
        jlat = item["location"]["lat"]
        jlng = item["location"]["lng"]
        js_sel = jname + ',' + str(jlat) + ',' + str(jlng)
        json_sel.append(js_sel)
                               
    max_page=int(a[0]/page_size)+1
            #防止并发过高，百度地图要求并发小于120
    time.sleep(1) 
        
    if page_num>max_page:
        np=False
        page_num=0
                
print("search complete")
print("output: ", str(self.loc))
print("total: ", str(a[0]))
print(json_sel)
print("")
                
#return json_sel,a[0]

URLError: <urlopen error [WinError 10061] 由于目标计算机积极拒绝，无法连接。>

In [ ]:
#######################  读 POI_list  从文件中  ######################
file_poi_list = ".\\os_file\\POI_list.csv"


    #filename = ".\\area_segment.csv"       #  读取搜索区域
f = open(file_poi_list, "rb")
Lat_lng = pd.read_csv(f,header = None, encoding = "gbk")
f.close()

POI_list = []
for k in range(0,len(Lat_lng)):
    a = [Lat_lng.iloc[k][0],float(Lat_lng.iloc[k][1]),float(Lat_lng.iloc[k][2])]
    POI_list.append(a)
    

In [ ]:
######################  把 POI_list 写入文件 #############################
file_poi_list = ".\\os_file\\POI_list.csv"
#def write_poi_list(POI_list, file_poi_list):
    
f = open(file_poi_list, 'a+')
for i in range(0, len(POI_list)):
    a = POI_list[i][0] + "," + str(POI_list[i][1]) + "," + str(POI_list[i][2])
    f.write(a)
    f.write("\n")
f.close()
 

In [ ]:
################################    汇总 POI_list    #####################################
POI_list_1 = []       # 暂存 POI_list,  后面要删除重复数据
class_1_path = ".\\POI_base\\POI\\"             #  大类路径
class_1_list = os.listdir(class_1_path)          #  大类list
for i in range(0, len(class_1_list)):   #  对每个大类的操作
    
    print("当前大类是： ", class_1_list[i])
    class_2_path = class_1_path + class_1_list[i]    #  小类路径
    class_2_list = os.listdir(class_2_path)          #  小类list
    print(class_2_list)
    
    for j in range(0, len(class_2_list)):     #  对每个小类的操作   
        
        print("当前小类是： ", class_2_list[j])
        file_name = class_2_path + "\\" +class_2_list[j]     # 取出文件名，对文件进行操作
        print(file_name)
        
        #a = read_POI(file_name)
        f = open(file_name,"rb")
        Lat_lng = pd.read_csv(f,header = None, encoding = "gbk")
        f.close()
        
        #POI_list = []
        for k in range(0,len(Lat_lng)):
            a = [Lat_lng.iloc[k][0],Lat_lng.iloc[k][1],Lat_lng.iloc[k][2]]
            POI_list_1.append(a)
        
        
POI_list = [item for item in POI_list_1 if POI_list_1.count(item) == 1]          #删除重复数据
len(POI_list)
        
#category_1 = 
#file_list[0]

In [ ]:
http://api.map.baidu.com/place/v2/search?query=%E4%B8%AD%E5%AD%A6&region=%E6%98%86%E6%98%8E&page_size=20&page_num=0&output=json&ak=Mep63CoO93mgoD3XlEFQ6MdTkNNugYKg